# Implementing GCN in PyTorch Geometric

[graph_nets исходный репо](https://github.com/dsgiitr/graph_nets)

## Dataset

Cora датасет из статей по МЛ, принадлежащихз одному из классов:

		Case_Based
		Genetic_Algorithms
		Neural_Networks
		Probabilistic_Methods
		Reinforcement_Learning
		Rule_Learning
		Theory

Статьи были отобраны таким образом, чтобы в окончательном корпусе каждая статья цитировала или цитировалась хотя бы одной другой статьей. Всего 2708 статей. Будем предсказывать класс, к которому принадлежит статья. 

## Working

[GCN статья](https://arxiv.org/abs/1609.02907) на всякий пожарный

$$\mathbf{x}_i^{(k)} = \sum_{j \in \mathcal{N}(i) \cup \{ i \}} \frac{1}{\sqrt{\deg(i)} \cdot \sqrt{deg(j)}} \cdot \left( \mathbf{\Theta} \cdot \mathbf{x}_j^{(k-1)} \right),$$

свойства соседних узлов сначала преобразуются весовой матрицей Θ, нормализуются по их степени и, наконец, суммируются. Все тоже самое:

- добавление петель в матрицу смежности
- линейные преобразования
- нормализация
- агрегация (суммирование)
- вернуть эмбеддинги

Шаги 1-2 обычно вычисляются до передачи сообщения. Шаги 3-5 делаются с помощью базового класса torch_geometric.nn.MessagePassing.

## Imports

In [1]:
import torch
import torch_geometric
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
from torch_geometric.datasets import Planetoid

In [2]:
torch.__version__

'1.5.1'

In [3]:
torch_geometric.__version__

'1.5.0'

### Loading the Dataset

In [4]:
dataset = Planetoid(root='dsets/Cora', name='Cora')

In [5]:
dataset[0]

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])

### The Graph Convolution Layer

[MessagePassing](https://pytorch-geometric.readthedocs.io/en/1.5.0/notes/create_gnn.html#creating-message-passing-networks)

[Пример сетки](https://pytorch-geometric.readthedocs.io/en/1.5.0/notes/introduction.html)

Свертка

In [6]:
class GraphConvolution(MessagePassing):
    def __init__(self, in_channels, out_channels, bias=True):
        super(GraphConvolution, self).__init__(aggr='add') # "Add" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels, bias=bias)

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)

        # Step 3: Compute normalization
        row, col = edge_index
        deg = degree(row, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # Step 4-6: Start propagating messages.
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x, norm=norm)

    def message(self, x_j, norm):
        # x_j has shape [E, out_channels]

        # Step 4: Normalize node features.
        return norm.view(-1, 1) * x_j

    def update(self, aggr_out):
        # aggr_out has shape [N, out_channels]

        # Step 6: Return new node embeddings.
        return aggr_out

Сеть

In [7]:
class Net(torch.nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(Net, self).__init__()
        self.conv1 = GraphConvolution(nfeat, nhid)
        self.conv2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [8]:
nfeat = dataset.num_node_features
nhid = 16
nclass = dataset.num_classes
dropout = 0.5

In [9]:
print(nfeat, nclass)

1433 7


## Training

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(nfeat, nhid, nclass, dropout).to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [11]:
model.eval()
_, pred = model(data).max(dim=1)
correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.8020
